In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_percentage_error

from petro_utils import compare_pred_results

import seaborn as sns

# Predicting Production with Machine Learning
The core of Petro.ai's machine learning pipeline are multivariate models that predict production based on geologic and engineering features of wells from a region of interest.  This notebook walks through a simplified workflow for building one of these models.  Your task is to run the simplified model, and understand the steps and the generated results.  You can use this as a start to improve the model, and demonstrate your experience and data science skills.

If need any clarification or have questions, feel free to reach out to Brendon at <brendon.hall@petro.ai>.

## Data
Petro.ai provides a download of well stats.  These are properties and attributes of the wells in the database.  These can be used as features to train predictive models. Well Stats contains one row per well with 100+ columns.  Don't worry, we won't use them all!

Here is a brief description of some of the columns in well stats that we might consider for modeling.
 * `totalDrainage` - predicted drainage area for a well (using Petro.ai drainage model)
 * `prodOil12mo` - cumulative 12 month oil production
 * `avgHzDistAnyZone` - average distance to nearby wells
 * `latitude`, `longitude` - surface location of well head
 * `angleFromSHMax` - angle between well azimuth and maximum horizontal stress direction


Well stats contains a raw download of all the wells in the collection.  There might be missing data, and we can transform these features before training a model.

As a first step, let's read the file `data/part2/well_stats_delaware.csv` into a pandas dataframe.

In [ ]:
# read the well stats data, save to a pandas dataframe
df = pd.read_csv('../data/part2/well_stats.csv') # answer

# print the number of rows and columns in this dataframe.
print(f"There are {df.shape[0]} rows and {df.shape[1]} columns in the raw well stats file.")

Let's rename the `totalDrainage` column to `drainage_area` to be more clear.

In [ ]:
# rename the 'totalDrainage' column to 'drainage_area'
df.rename(columns={'totalDrainage':'drainage_area'}, inplace=True)

## Data Exploration
For an initial model we'll start with a few features from well stats described above:

- drainage area
- totalProppantByPerfLength
- avgHzDistAnyZone
- latitude
- longitude
- angleFromSHMax

There are many more we can try, but this will get us started.  For the first model,
let's try and predict 12 month cumulative oil production `prodOil12mo` directly.

First, plot the distribution of these features with respect to the `prodOil12mo` target.

In [ ]:
feature_names = ['drainage_area', 'totalProppantByPerfLength', 'avgHzDistAnyZone', 'latitude', 'longitude', 'angleFromSHMax']

target = 'prodOil12mo'

g = sns.pairplot(df, x_vars=feature_names, y_vars=target, hue='interval')
plt.show()

Did you notice any significant trends in these features?  Any outliers or potential issues? There are a number of wells with zero drainage, we'll filter those out.

Feel free to experiment with other types of plots, features, etc.

**[Debug Question]** In the next cell, we want to find the 12 mo. cumulative oil production for a specific well with `wellAPI` = 4200000036.  There is a bug in this code though.  Can you fix it to output the correct production amount?

In [ ]:
well_production = df[df['wellAPI'] == "4200000036"]["prodOil12mo"].values[0]

print("The production of the well with API 4200000036 is:", well_production)

## Data Filtering

Remove any wells with zero drainage_area from the dataset.

In [ ]:
# Remove wells with zero drainage area
df = df[df['drainage_area'] > 0].copy()

Now remove any wells that are missing data for any of the features or target from the dataset.  How many wells are left in the dataset?

In [ ]:
# Remove wells missing features or production
df.dropna(subset=feature_names+[target], inplace=True)
print(f"There are {df.shape[0]} rows in the cleaned well stats file.")

## Prepare Training Data

Now the dataset is ready to train a model.  Extract the features, $X$, and the "target" to be predicted, $y$, from the dataframe.  For now, we'll randomly select a third of the wells and set them aside to test the accuracy of the model.  We'll use the remainder as training data.

In [ ]:
from sklearn.model_selection import train_test_split

X = df[feature_names]
y = df[target]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42)

## Train Random Forest Model

Let's start simple. We'll build a simple random forest model and take a look at the prediction accuracy. 

In [ ]:
rf_model = RandomForestRegressor()

rf_model.fit(X_train, y_train)

y_train_pred = rf_model.predict(X_train)
y_test_pred = rf_model.predict(X_test)



## Model Evaluation
What is the $R^2$ score of this regression?  What is the error (or accuracy) of the result?

In [ ]:
# mean_absolute_percentage_error(y_test, y_test_pred)
r2 = r2_score(y_train, y_train_pred)
acc =  1- mean_absolute_percentage_error(y_train, y_train_pred)
print(f"Train data: the r2 score is {r2:.3f}, the accuracy is {acc:.3f}")

r2 = r2_score(y_test, y_test_pred)
acc =  1- mean_absolute_percentage_error(y_test, y_test_pred)
print(f"Test data: the r2 score is {r2:.3f}, the accuracy is {acc:.3f}")

Use the utility `compare_pred_results` function to plot the predicted vs. actual production.

In [ ]:
compare_pred_results(y_train, y_train_pred, y_test, y_test_pred, scale_factor=1e3, title="Predicted vs. Actual 12 mo. cumulative production",
                     xlabel="Actual 12 mo. oil [Mbbl]", ylabel="Predicted 12 mo. oil [Mbbl]")

The accuracy on the training data is significantly higher than the test data.  Random forest models are notorious for over-fitting. Do you notice any other interesting trends in these results?  There is plenty of room for improvement.  So far, we've guided you through building a simple model.  Now you can unleash your own skills and creativity to build a model that improves on these results.  Feel free to improve on the random forest model and/or use another library.  Be prepared to discuss your approach, what you tried, the pros/cons of your modeling approach, etc.

Some things you might try:
- hyperparameter tuning
- additional/different features from well stats
- feature selection/engineering
- proper cross-validation

You might also consider doing some error analysis.  Is the model making any big mistakes?  Is it due to bad training data or is there a systematic bias?  Can you explain why the model is making the decisions that it is?  Are some features more important than others in terms of making decisions?

Prepare a short presentation that demonstrates your data exploration, modeling and analysis efforts. Tell a story with the data. What are your insights on the problem/dataset?  Be prepared to answer questions on your approach and assumptions.  Think about how you would continue this work if you had more time and resources.  You'll give a brief presentation to a couple members of the Petro.ai team, and then we'll discussion around your experience with this task.